In [105]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [121]:
df = pd.read_csv('data/england-premier-league-matches-2017-to-2018-stats.csv')

In [62]:
df.columns

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

In [85]:
df.describe()

,timestamp,attendance,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,...,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no
count,3.800000e+02,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,...,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.514574e+09,38316.707895,19.500000,1.509105,1.105105,1.626000,1.112500,1.531579,1.147368,2.678947,...,3.228684,3.055632,4.425474,4.586474,1.263368,1.880211,3.243632,6.732974,1.878553,1.827105
std,6.827494e+06,17802.226732,10.980313,0.755472,0.793324,0.517686,0.534331,1.340087,1.177793,1.665125,...,1.197882,2.272880,1.619347,2.986443,0.209504,0.404690,1.020293,2.885210,0.356175,0.347720
min,1.502477e+09,10242.000000,1.000000,0.000000,0.000000,0.950000,0.580000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.509199e+09,24102.500000,10.000000,1.105000,0.607500,1.330000,0.740000,1.000000,0.000000,1.000000,...,2.830000,1.637500,3.330000,2.547500,1.187500,1.610000,2.500000,4.500000,1.750000,1.690000
50%,1.514175e+09,31545.000000,19.500000,1.400000,0.865000,1.420000,0.865000,1.000000,1.000000,3.000000,...,3.300000,2.265000,3.715000,3.545000,1.270000,1.870000,3.200000,6.450000,1.910000,1.830000
75%,1.520089e+09,53272.250000,29.000000,2.000000,1.640000,2.027500,1.522500,2.000000,2.000000,4.000000,...,3.905000,2.955000,5.142500,5.827500,1.380000,2.150000,3.927500,8.600000,2.050000,2.000000
max,1.526220e+09,83222.000000,38.000000,3.000000,3.000000,2.630000,2.630000,7.000000,6.000000,9.000000,...,11.000000,9.990000,9.990000,9.990000,1.610000,2.900000,6.000000,15.000000,3.130000,2.650000


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 66 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   timestamp                            380 non-null    int64  
 1   date_GMT                             380 non-null    object 
 2   status                               380 non-null    object 
 3   attendance                           380 non-null    int64  
 4   home_team_name                       380 non-null    object 
 5   away_team_name                       380 non-null    object 
 6   referee                              373 non-null    object 
 7   Game Week                            380 non-null    int64  
 8   Pre-Match PPG (Home)                 380 non-null    float64
 9   Pre-Match PPG (Away)                 380 non-null    float64
 10  home_ppg                             380 non-null    float64
 11  away_ppg                        

In [64]:
data = df.copy()

In [67]:
df.shape

(380, 66)

In [68]:
df.isnull().sum()[data.isnull().sum() > 0]

referee                     7
home_team_goal_timings     90
away_team_goal_timings    136
dtype: int64

In [69]:
data = df.drop(['referee', 'home_team_goal_timings', 'away_team_goal_timings'], axis=1)

In [71]:
data.shape

(380, 63)

In [72]:
data.isnull().sum().sum()

0

In [63]:
target_columns = ['home_team_goal_count', 'away_team_goal_count']

In [91]:
non_numeric_columns = ['date_GMT', 'status', 'home_team_name', 'away_team_name', 'referee',
                        'home_team_goal_timings', 'away_team_goal_timings', 'stadium_name']

X = df.drop(target_columns + non_numeric_columns, axis=1)
y = df[target_columns]

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [93]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [96]:
def create_sequences(data, target, sequence_length=10):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length + 1):
        seq = data[i:i+sequence_length]
        label = target[i+sequence_length-1]
        sequences.append(seq)
        targets.append(label)
    return np.array(sequences), np.array(targets)

In [97]:
sequence_length = 10
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train.values, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test.values, sequence_length)

In [98]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dense(2, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [112]:
history = model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/50
9/9 [==============================] - 0s 21ms/step - loss: 0.0294 - val_loss: 0.6611
Epoch 2/50
9/9 [==============================] - 0s 9ms/step - loss: 0.0274 - val_loss: 0.6624
Epoch 3/50
9/9 [==============================] - 0s 13ms/step - loss: 0.0257 - val_loss: 0.6487
Epoch 4/50
9/9 [==============================] - 0s 8ms/step - loss: 0.0236 - val_loss: 0.6569
Epoch 5/50
9/9 [==============================] - 0s 10ms/step - loss: 0.0212 - val_loss: 0.6486
Epoch 6/50
9/9 [==============================] - 0s 12ms/step - loss: 0.0193 - val_loss: 0.6487
Epoch 7/50
9/9 [==============================] - 0s 10ms/step - loss: 0.0179 - val_loss: 0.6514
Epoch 8/50
9/9 [==============================] - 0s 14ms/step - loss: 0.0168 - val_loss: 0.6540
Epoch 9/50
9/9 [==============================] - 0s 9ms/step - loss: 0.0153 - val_loss: 0.6475
Epoch 10/50
9/9 [==============================] - 0s 9ms/step - loss: 0.0142 - val_loss: 0.6515
Epoch 11/50
9/9 [================

In [100]:
y_pred = model.predict(X_test_seq)

3/3 [==============================] - 0s 3ms/step


In [109]:
rmse = np.sqrt(np.mean((y_test_seq - y_pred)**2))
print(f'Root Mean Squared Error on Test Set: {rmse}')

Root Mean Squared Error on Test Set: 0.6566698497246275


In [108]:
y_pred_discrete = np.round(y_pred).astype(int)
confusion_matrix(y_test_seq.flatten(), y_pred_discrete.flatten())

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, 23, 14,  0,  0,  0,  0,  0,  0],
       [ 0,  7, 36,  5,  1,  0,  0,  0,  0],
       [ 0,  1,  8, 19,  1,  0,  0,  0,  0],
       [ 0,  0,  1,  5,  4,  1,  0,  0,  0],
       [ 0,  0,  0,  1,  2,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0]], dtype=int64)

In [107]:
accuracy_score(y_test_seq.flatten(), y_pred_discrete.flatten())

0.6119402985074627